In [ ]:
!pip install owid-catalog

In [ ]:
from owid.catalog import RemoteCatalog
# find the default OWID catalog and fetch the catalog index over HTTPS
catalog = RemoteCatalog(channels=('garden', 'meadow', 'open_numbers'))
catalog.

In [ ]:
# All namespaces = main sources
catalog.datasets["namespace"].unique()

In [ ]:

# GET LATEST VERSIONS of datasets
import pandas as pd
import numpy as np

df = catalog.datasets

# Group the data by 'namespace, dataset' and get latest version
df = df.groupby(['namespace', 'dataset'])['version'].max()

df = df.reset_index()
df


Population

In [ ]:
# Population
df_pop = df[df["dataset"].str.contains("population")]
df_pop

data_pop = catalog.find_latest(namespace='gapminder', dataset='population')
data_pop


In [ ]:

# GH Emmissions filter
df_gh = df[df["dataset"].str.contains("gh|green|house|carbon|co2|emission")]
df_gh


In [ ]:
# NRJ filter
df_nrj = df[df["dataset"].str.contains("nrj|energy|consumption|final")]
df_nrj


In [ ]:
#  !!!! FROM SHIFT DATA PORTAL (UP TO 2016)
sdp = catalog.find_latest(namespace='shift')
print(dataset.metadata.dataset)
dataset.sort_values('year', ascending=False).head()

In [ ]:
# DEFAULT LATEST EIA

df = catalog.find_latest(namespace='eia', dataset='energy_consumption')
print(df.metadata.dataset)
df